In [1]:
import os
import conda
import pandas as pd
import numpy as np
import json
# from shapely.geometry import Polygon as Poly

pd.options.display.max_columns = 250

conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib

import matplotlib.pyplot as plt
import matplotlib.cm
import matplotlib.colors as colors

import geopandas
from geopandas.tools import sjoin
import geoplot as gplt
import geoplot.crs as gcrs

import folium

In [ ]:
# %matplotlib inline

Take Land Cover table from EnviroAtlas, regarding land cover, and reconciliate at HUC8 level since chl-a data for TSI estimation is only available at HUC8 spatial scale. Additionaly, the work of Lee and Bakshi, 2019(Energy-Water-CO2 Nexus of Fossil Fuel Based Power Generation, book carbon management) support the use of HUC8 scale for Techno-Ecological synergy (TES) analysis.

The fields of the databse are described below:

| Field Name |                                           Data Layer Name                                           |
|:----------:|:---------------------------------------------------------------------------------------------------:|
|   N_INDEX  |                                      Percent natural land cover                                     |
|    PAGT    | Percentage of WBD 12-digit Hydrologic Unit Land area that is classified as agriculture (21, 81, 82) |
|   PFOR90   |                                  Percent forest and woody wetlands                                  |
|   PWETL95  |                                 Percent emergent herbaceous wetlands                                |
|    PFOR    |                                            Percent forest                                           |
|    PWETL   |                                           Percent wetlands                                          |
|    PDEV    |                                        Percent developed area                                       |
|    PAGC    |                                           Percent cropland                                          |
|    PAGP    |                                           Percent pasture                                           |


HUC12 is transformed in HUC8 for data spatial consistency deleting the last 4 digits of the code (Federal Standards and Procedures for the National Watershaed Boundary Dataset (WBD), USGS, 2013). 

To restimate the land cover percentajes the values of each item for the same HUC8 watershed are summed, and the percentajes recalculated again over the new total value for HUC8

In [12]:
# File loaded again to avoid overwritting of the LandCover_df variable
LandCover_df = pd.read_csv('LandCover/National_metric_tables_in_CSV/CONUS_metrics_May2019_CSV/LandCover.csv') 

# Convert HUC12 to HUC8
LandCover_df['HUC_8'] = LandCover_df['HUC_12'].map(lambda x: str(x)[:-4])
LandCover_df = LandCover_df.drop('HUC_12',1) #0 for rows, 1 for columns

LandCover_df['HUC_8'] = LandCover_df['HUC_8'].astype(int)
LandCover_df['Total'] = 100

LandCover_df

,N_INDEX,PFOR,PWETL,PDEV,PAGT,PAGP,PAGC,PFOR90,PWETL95,HUC_8,Total
0,100.000000,81.643204,11.392500,0.000000,0.000000,0.000000,0.000000,92.245598,0.789988,1010002,100
1,100.000000,74.108200,12.300500,0.000000,0.000000,0.000000,0.000000,86.143501,0.265201,1010002,100
2,100.000000,78.816101,13.675100,0.000000,0.000000,0.000000,0.000000,92.095398,0.395778,1010002,100
3,100.000000,72.776901,7.579820,0.000000,0.000000,0.000000,0.000000,80.097702,0.259059,1010002,100
4,100.000000,74.281403,13.116500,0.000000,0.000000,0.000000,0.000000,81.182098,6.215810,1010002,100
5,99.839203,80.429001,5.992480,0.160756,0.000000,0.000000,0.000000,86.216202,0.205273,1010002,100
6,100.000000,82.489899,12.551700,0.000000,0.000000,0.000000,0.000000,94.915604,0.125930,1010002,100
7,100.000000,79.267097,11.043100,0.000000,0.000000,0.000000,0.000000,89.944801,0.365425,1010002,100
8,99.994400,80.136299,9.918930,0.000000,0.005647,0.000000,0.005647,87.546501,2.508770,1010002,100
9,99.774200,83.703796,9.742270,0.225844,0.000000,0.000000,0.000000,93.268997,0.177132,1010002,100


In [13]:
# Grouped and values sumed, but not percetages yet!
LandCover_dfGrouped = LandCover_df.groupby(['HUC_8']).sum()
LandCover_dfGrouped

,N_INDEX,PFOR,PWETL,PDEV,PAGT,PAGP,PAGC,PFOR90,PWETL95,Total
HUC_8,,,,,,,,,,
1010002,4097.643600,3321.519814,282.046980,2.071450,0.284989,0.088163,0.196827,3583.079910,20.487259,4100
1010003,2291.215496,1733.613396,297.340113,29.957569,178.826970,39.092481,139.734379,2012.386116,18.567774,2500
1010004,5987.979889,4391.402512,917.224541,94.488543,517.531725,77.320543,440.211255,5279.747490,28.879693,6600
1010005,1033.428600,711.407206,244.454398,46.463798,220.107512,30.664473,189.442966,950.056610,5.804847,1300
1010006,2194.817802,1476.533783,317.074897,5.078495,0.103808,0.069891,0.033917,1775.495705,18.113087,2200
1010007,3599.394402,2724.958996,331.773067,0.500214,0.105245,0.039239,0.066006,3034.023071,22.709279,3600
1010008,1202.609093,1020.640102,80.317040,34.136468,63.254378,11.198781,52.055658,1095.480297,5.476792,1300
1010009,445.784084,323.178200,76.634000,32.029460,122.186329,28.046620,94.139681,394.393005,5.419244,600
1010010,562.933506,389.770494,138.588140,36.024910,301.041601,38.929170,262.112300,523.732201,4.626198,900


In [14]:
# Percetages calculation
LandCover_dfHUC8 = LandCover_dfGrouped
LandCover_dfHUC8.update(LandCover_dfHUC8.iloc[:, :-1].div(LandCover_dfHUC8.Total, 0))
LandCover_dfHUC8.update(LandCover_dfHUC8.iloc[:, :-1].mul(100))
LandCover_dfHUC8['Total'] = LandCover_dfHUC8['PFOR'] + LandCover_dfHUC8['PWETL'] + LandCover_dfHUC8['PDEV'] + LandCover_dfHUC8['PAGC'] + LandCover_dfHUC8['PAGP']
LandCover_dfHUC8

,N_INDEX,PFOR,PWETL,PDEV,PAGT,PAGP,PAGC,PFOR90,PWETL95,Total
HUC_8,,,,,,,,,,
1010002,99.942527,81.012678,6.879195,0.050523,0.006951,0.002150,0.004801,87.392193,0.499689,87.949347
1010003,91.648620,69.344536,11.893605,1.198303,7.153079,1.563699,5.589375,80.495445,0.742711,89.589518
1010004,90.726968,66.536402,13.897342,1.431645,7.841390,1.171523,6.669868,79.996174,0.437571,89.706779
1010005,79.494508,54.723631,18.804184,3.574138,16.931347,2.358806,14.572536,73.081278,0.446527,94.033295
1010006,99.764446,67.115172,14.412495,0.230841,0.004719,0.003177,0.001542,80.704350,0.823322,81.763227
1010007,99.983178,75.693305,9.215919,0.013895,0.002923,0.001090,0.001833,84.278419,0.630813,84.926042
1010008,92.508392,78.510777,6.178234,2.625882,4.865721,0.861445,4.004281,84.267715,0.421292,92.180619
1010009,74.297347,53.863033,12.772333,5.338243,20.364388,4.674437,15.689947,65.732168,0.903207,92.337993
1010010,62.548167,43.307833,15.398682,4.002768,33.449067,4.325463,29.123589,58.192467,0.514022,96.158335


In [16]:
# Save the new database
LandCover_dfHUC8.to_csv('DatabasesClean/LandCover_HUC8.csv', index=True)

## 2017 Agricultural Census

The 2017 Agricultural Census is used to determine the distribution of crops on croplands. However, the data provided by the Agricultural Census have a spatial resolution of HUC6 corresponding to each hydrological unit HUC8 to determine the distribution of crops in each portion of crop land corresponding to each hydrological unit HUC8. Area is in acres (2017 census of Agriculture. Watersheds. U.S. Department of Agriculture, 2019).

In [2]:
# Load csv file
AgriCensus_df = pd.read_csv('AgriCensus2017Watershed/wate_all_tablesCLEANED_TRASPOSED.csv')

AgriCensus_df

,HUC_6,Year,Corn,Soybeans,Small grains,Cotton,Rice,Vegetables,Orchards,Greenhouse,Other
0,H010100,2017,1833.0,1254.0,37038.0,NaN,NaN,51854.0,137.0,11.0,22151.0
1,H010200,2017,9257.0,NaN,1170.0,NaN,NaN,3453.0,412.0,58.0,33209.0
2,H010300,2017,14719.0,NaN,293.0,NaN,NaN,758.0,356.0,195.0,59311.0
3,H010400,2017,4002.0,NaN,NaN,NaN,NaN,1069.0,1059.0,66.0,24189.0
4,H010500,2017,163.0,NaN,NaN,NaN,NaN,1158.0,435.0,154.0,55529.0
5,H010600,2017,3639.0,NaN,170.0,NaN,NaN,3813.0,857.0,2181.0,38624.0
6,H010700,2017,4894.0,NaN,NaN,NaN,NaN,4409.0,2983.0,1417.0,41120.0
7,H010801,2017,19289.0,NaN,NaN,NaN,NaN,1685.0,980.0,208.0,99533.0
8,H010802,2017,13153.0,457.0,523.0,NaN,NaN,15501.0,2829.0,4282.0,62847.0
9,H010900,2017,4433.0,NaN,6.0,NaN,NaN,7313.0,1328.0,3782.0,38678.0


In [3]:
# AgriCensus_df.dropna(axis='index', how ='any', subset=['HUC_6'])
# AgriCensus_df.dropna(axis='index', how ='any', subset=['HUC_6']).to_csv('AgriCensus2017Watershed/wate_all_tablesCLEANED_TRASPOSED.csv',index=False)

In [5]:
# Percetages calculation
AgriCensus_df['Total'] = AgriCensus_df[['Corn', 'Soybeans', 'Small grains', 'Cotton', 'Rice', 'Vegetables', 'Orchards', 'Greenhouse', 'Other']].sum(axis=1)
# AgriCensus_df.iloc[:, 1:].astype(float)

AgriCensus_df

,HUC_6,Year,Corn,Soybeans,Small grains,Cotton,Rice,Vegetables,Orchards,Greenhouse,Other,Total
0,H010100,2017,1833.0,1254.0,37038.0,NaN,NaN,51854.0,137.0,11.0,22151.0,114278.0
1,H010200,2017,9257.0,NaN,1170.0,NaN,NaN,3453.0,412.0,58.0,33209.0,47559.0
2,H010300,2017,14719.0,NaN,293.0,NaN,NaN,758.0,356.0,195.0,59311.0,75632.0
3,H010400,2017,4002.0,NaN,NaN,NaN,NaN,1069.0,1059.0,66.0,24189.0,30385.0
4,H010500,2017,163.0,NaN,NaN,NaN,NaN,1158.0,435.0,154.0,55529.0,57439.0
5,H010600,2017,3639.0,NaN,170.0,NaN,NaN,3813.0,857.0,2181.0,38624.0,49284.0
6,H010700,2017,4894.0,NaN,NaN,NaN,NaN,4409.0,2983.0,1417.0,41120.0,54823.0
7,H010801,2017,19289.0,NaN,NaN,NaN,NaN,1685.0,980.0,208.0,99533.0,121695.0
8,H010802,2017,13153.0,457.0,523.0,NaN,NaN,15501.0,2829.0,4282.0,62847.0,99592.0
9,H010900,2017,4433.0,NaN,6.0,NaN,NaN,7313.0,1328.0,3782.0,38678.0,55540.0


In [6]:
AgriCensus_df.update(AgriCensus_df.iloc[:, 2:-1].div(AgriCensus_df.Total, 0))
AgriCensus_df.update(AgriCensus_df.iloc[:, 2:-1].mul(100))
AgriCensus_df['Sum'] = AgriCensus_df.iloc[:, 2:-1].sum(axis=1)
AgriCensus_df

,HUC_6,Year,Corn,Soybeans,Small grains,Cotton,Rice,Vegetables,Orchards,Greenhouse,Other,Total,Sum
0,H010100,2017,1.603983,1.097324,32.410438,NaN,NaN,45.375313,0.119883,0.009626,19.383433,114278.0,100.0
1,H010200,2017,19.464244,NaN,2.460102,NaN,NaN,7.260455,0.866292,0.121954,69.826952,47559.0,100.0
2,H010300,2017,19.461339,NaN,0.387402,NaN,NaN,1.002221,0.470700,0.257827,78.420510,75632.0,100.0
3,H010400,2017,13.170973,NaN,NaN,NaN,NaN,3.518183,3.485272,0.217212,79.608359,30385.0,100.0
4,H010500,2017,0.283779,NaN,NaN,NaN,NaN,2.016052,0.757325,0.268111,96.674733,57439.0,100.0
5,H010600,2017,7.383735,NaN,0.344940,NaN,NaN,7.736791,1.738901,4.425371,78.370262,49284.0,100.0
6,H010700,2017,8.926910,NaN,NaN,NaN,NaN,8.042245,5.441147,2.584682,75.005016,54823.0,100.0
7,H010801,2017,15.850281,NaN,NaN,NaN,NaN,1.384609,0.805292,0.170919,81.788898,121695.0,100.0
8,H010802,2017,13.206884,0.458872,0.525143,NaN,NaN,15.564503,2.840590,4.299542,63.104466,99592.0,100.0
9,H010900,2017,7.981635,NaN,0.010803,NaN,NaN,13.167087,2.391069,6.809507,69.639899,55540.0,100.0


In [8]:
# Delete H from HUC_6 columns
AgriCensus_df['HUC_6'] = AgriCensus_df['HUC_6'].map(lambda x: str(x)[1:])
AgriCensus_df

,HUC_6,Year,Corn,Soybeans,Small grains,Cotton,Rice,Vegetables,Orchards,Greenhouse,Other,Total,Sum
0,010100,2017,1.603983,1.097324,32.410438,NaN,NaN,45.375313,0.119883,0.009626,19.383433,114278.0,100.0
1,010200,2017,19.464244,NaN,2.460102,NaN,NaN,7.260455,0.866292,0.121954,69.826952,47559.0,100.0
2,010300,2017,19.461339,NaN,0.387402,NaN,NaN,1.002221,0.470700,0.257827,78.420510,75632.0,100.0
3,010400,2017,13.170973,NaN,NaN,NaN,NaN,3.518183,3.485272,0.217212,79.608359,30385.0,100.0
4,010500,2017,0.283779,NaN,NaN,NaN,NaN,2.016052,0.757325,0.268111,96.674733,57439.0,100.0
5,010600,2017,7.383735,NaN,0.344940,NaN,NaN,7.736791,1.738901,4.425371,78.370262,49284.0,100.0
6,010700,2017,8.926910,NaN,NaN,NaN,NaN,8.042245,5.441147,2.584682,75.005016,54823.0,100.0
7,010801,2017,15.850281,NaN,NaN,NaN,NaN,1.384609,0.805292,0.170919,81.788898,121695.0,100.0
8,010802,2017,13.206884,0.458872,0.525143,NaN,NaN,15.564503,2.840590,4.299542,63.104466,99592.0,100.0
9,010900,2017,7.981635,NaN,0.010803,NaN,NaN,13.167087,2.391069,6.809507,69.639899,55540.0,100.0


In [10]:
# Save the new database
AgriCensus_df.to_csv('DatabasesClean/Agricensus.csv', index=False)

## Vegetation nutrient uptake

To determine the nutrients uptake of each type of crop data from USDA (2009) is considered (Waste Management Field Handbook). 

For croplands, the land is divided in the portion occupied by corn, soybeans, small grains, cotton, rice,	vegetables, orchards, greenhouse and other crops (namely oil crops, sugar crops, and fruits). 

For pasture lands the average nutrient uptake and crop yield including the following plants is considered:

    -Alfalfa
    -Switchgrass
    -Wheatgrass

For forests lands the nutrient uptake and crop yield of Northern hardwoods is considered

For corn, soybeans, cotton, rice and orchards their specific nutrient uptake values are used. For small grains, vegetables, greenhouse crops, pasture crops and forest average values including the most representatve species are used. (USDA, 2009. Waste Management Field Handbook)b

In [33]:
PlantNutrientUptake_df =  pd.read_csv('PlantNutrientUptake/PlantNutrientUptake.csv', index_col='Crop')
PlantNutrientUptake_df

,ConversionFactor,YieldAcre,PoundsAcre,N,P,K,Ca,Mg,S,Cu,Mn,Zn
Crop,,,,,,,,,,,,
Barley,48.0,50.00,2400.0,1.82,0.34,0.43,0.05,0.10,0.1600,0.0016,0.0016,0.0031
BarleyStraw,2000.0,1.00,2000.0,0.75,0.11,1.25,0.40,0.10,0.2000,0.0005,0.0160,0.0025
Buckwheat,48.0,30.00,1440.0,1.65,0.31,0.45,0.09,NaN,NaN,0.0009,0.0034,NaN
BuckwheatStraw,2000.0,0.50,1000.0,0.78,0.05,2.26,1.40,NaN,0.0100,NaN,NaN,NaN
Corn,56.0,120.00,6720.0,1.61,0.28,0.40,0.02,0.10,0.1200,0.0007,0.0011,0.0018
CornStraw,2000.0,1.50,3000.0,1.11,0.20,1.34,0.29,0.22,0.1600,0.0005,0.0166,0.0033
Oats,32.0,80.00,2560.0,1.95,0.34,0.49,0.08,0.12,0.2000,0.0012,0.0047,0.0020
OatsStraw,2000.0,2.00,4000.0,0.63,0.16,1.66,0.20,0.20,0.2300,0.0008,0.0030,0.0072
Rice,45.0,5500.00,247500.0,1.39,0.24,0.23,0.08,0.11,0.0800,0.0030,0.0022,0.0019


For pasture lands the average nutrient uptake and crop yield including the following plants is considered:

    -Alfalfa
    -Switchgrass
    -Wheatgrass

In [34]:
Pasture_df = PlantNutrientUptake_df.loc[['Alfalfa','Switchgrass','Wheatgrass']]
Pasture_df = Pasture_df.mean(axis=0)
Pasture_df.to_csv('DatabasesClean/PasturePlantNutrientUptake.csv', index=False)
Pasture_df

/home/emh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


ConversionFactor    2000.000000
YieldAcre              2.666667
PoundsAcre          5333.333333
N                      1.606667
P                      0.196667
K                      2.150000
Ca                     0.680000
Mg                     0.250000
S                      0.175000
Cu                     0.000800
Mn                     0.005500
Zn                     0.005300
dtype: float64

For forests lands the nutrient uptake and crop yield of Northern hardwoods is considered

In [35]:
Forests_df = PlantNutrientUptake_df.loc[['Northern hardwoods']]
Forests_df = Forests_df.mean(axis=0)
Forests_df.to_csv('DatabasesClean/ForestsPlantNutrientUptake.csv', index=False)
Forests_df

/home/emh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


ConversionFactor      2000.00
YieldAcre               50.00
PoundsAcre          100000.00
N                        0.20
P                        0.02
K                        0.10
Ca                       0.29
Mg                        NaN
S                         NaN
Cu                        NaN
Mn                        NaN
Zn                        NaN
dtype: float64

For small grains the average nutrient uptake and crop yield including the following plants is considered:

    -Wheat
    -WheatStraw
    -Oats
    -OatsStraw
    -Barley
    -BarleyStraw
    -Rye
    -RyeStraw

In [36]:
SmallGrains_df = PlantNutrientUptake_df.loc[['Wheat','WheatStraw','Oats','OatsStraw','Barley','BarleyStraw','Rye','RyeStraw']]
SmallGrains_df = SmallGrains_df.mean(axis=0)
SmallGrains_df.to_csv('DatabasesClean/SmallGrainsPlantNutrientUptake.csv', index=False)
SmallGrains_df

/home/emh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


ConversionFactor    1024.500000
YieldAcre             25.750000
PoundsAcre          2630.000000
N                      1.310000
P                      0.252500
K                      0.812500
Ca                     0.170000
Mg                     0.140000
S                      0.201250
Cu                     0.004613
Mn                     0.006525
Zn                     0.003300
dtype: float64

For vegetables and greehouse the average nutrient uptake and crop yield including the following plants is considered:

    -Bell peppers
    -Beans
    -Cabbage
    -Carrots
    -Cassava
    -Celery
    -Cucumbers
    -Lettuce
    -Onions
    -Peas
    -Potatoes
    -Snap beans
    -Sweet corn
    -Sweet potatoes
    -Table beets
    -Cantaloupe

In [37]:
Vegetables_Greenhouse_df = PlantNutrientUptake_df.loc[['Bell peppers','Beans','Cabbage','Carrots','Cassava','Celery','Cucumbers','Lettuce',
                                                       'Onions','Peas','Potatoes','Snap beans','Sweet corn','Sweet potatoes','Table beets',
                                                       'Cantaloupe']]
Vegetables_Greenhouse_df = Vegetables_Greenhouse_df.mean(axis=0)
Vegetables_Greenhouse_df.to_csv('DatabasesClean/Vegetables_GreenhousePlantNutrientUptake.csv', index=False)
Vegetables_Greenhouse_df

/home/emh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """


ConversionFactor     1875.062500
YieldAcre            1104.062500
PoundsAcre          21718.750000
N                       0.744375
P                       0.138125
K                       0.505000
Ca                      0.071000
Mg                      0.084286
S                       0.096000
Cu                      0.000275
Mn                      0.001175
Zn                      0.001040
dtype: float64

For other crops, the average nutrient uptake and crop yield considered includes the most representative crop types not considered in other items, namely oil crops, sugar crops, and fruits:

    -Flax
    -FlaxStraw
    -Oil palm
    -Oil palmStraw
    -Peanuts
    -PeanutsStraw
    -Rapeseed
    -RapeseedStraw
    -Soybeans
    -SoybeansStraw
    -Sunflower
    -SunflowerStraw
    -Apples
    -Bananas
    -Coconuts
    -Grapes
    -Oranges
    -Peaches
    -Pineapple
    -Tomatoes
    -Sugarcane

In [38]:
OtherCrops_df = PlantNutrientUptake_df.loc[['Flax','FlaxStraw','Oil palm','Oil palmStraw','Peanuts','PeanutsStraw','Rapeseed','RapeseedStraw',
                                            'Soybeans','SoybeansStraw','Sunflower','SunflowerStraw','Apples','Bananas','Coconuts','Grapes',
                                            'Oranges','Peaches','Pineapple','Tomatoes','Sugarcane']]
OtherCrops_df = OtherCrops_df.mean(axis=0)
OtherCrops_df.to_csv('DatabasesClean/OtherCropsPlantNutrientUptake.csv', index=False)
OtherCrops_df

/home/emh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """


ConversionFactor     1248.571429
YieldAcre            4284.838571
PoundsAcre          18290.952381
N                       1.990476
P                       0.349524
K                       1.180000
Ca                      0.415000
Mg                      0.219048
S                       0.194667
Cu                      0.001067
Mn                      0.006575
Zn                      0.002614
dtype: float64

## Wetlands nutrients regulation

The phosporus uptake due to wetlands is considered as 0.77 gP·m-2·year-1 based in the data reported by Kadlec, 2016 (Kadlec, R.H., 2016
 Large Constructed Wetlands for Phosphorus Control: A Review. Water, 8, 243)